# Funding rates analysis across different exchanges

### Merge all BTC pairs and graph the funding rates

In [107]:
# Import needed libraries
import pandas as pd
import plotly.graph_objects as go
from glob import glob
import numpy as np
import plotly.express as px

In [2]:
# List all exchanges
exchanges = ["Binance", "BitMex", "Bybit", "Deribit", "Huobi", "OKX"]

In [17]:
# Step 1: Read and process data
def read_and_resample_csv(file_path, resample_period='12H'):
    df = pd.read_csv(file_path, parse_dates=['date'])
    df = df[["date", "funding_rate"]]
    df.set_index('date', inplace=True)
    df = df.tz_localize(None)
    df_resampled = df.resample(resample_period).mean()
    return df_resampled

In [49]:
exchange_data = {}
for exchange in exchanges:
    coin_exchange = "BTC_" + exchange
    exchange_data[coin_exchange] = read_and_resample_csv("data/" + coin_exchange + ".csv")

In [50]:
# Step 2: Merge data
merged_df = pd.concat(exchange_data, axis=1)
merged_df.columns = [f"{name}_{col}" for name in exchange_data.keys() for col in exchange_data[name].columns]
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3420 entries, 2019-09-10 00:00:00 to 2024-05-15 12:00:00
Freq: 12H
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   BTC_Binance_funding_rate  3419 non-null   float64
 1   BTC_BitMex_funding_rate   1836 non-null   float64
 2   BTC_Bybit_funding_rate    3024 non-null   float64
 3   BTC_Deribit_funding_rate  2322 non-null   float64
 4   BTC_Huobi_funding_rate    2002 non-null   float64
 5   BTC_OKX_funding_rate      2002 non-null   float64
dtypes: float64(6)
memory usage: 187.0 KB


In [51]:
merged_df.head()

,BTC_Binance_funding_rate,BTC_BitMex_funding_rate,BTC_Bybit_funding_rate,BTC_Deribit_funding_rate,BTC_Huobi_funding_rate,BTC_OKX_funding_rate
date,,,,,,
2019-09-10 00:00:00,0.0001,NaN,NaN,NaN,NaN,NaN
2019-09-10 12:00:00,0.0001,NaN,NaN,NaN,NaN,NaN
2019-09-11 00:00:00,0.0001,NaN,NaN,NaN,NaN,NaN
2019-09-11 12:00:00,0.0001,NaN,NaN,NaN,NaN,NaN
2019-09-12 00:00:00,0.0001,NaN,NaN,NaN,NaN,NaN


In [52]:
# Scale back some of the exchanges by 100 (to convert from percentage to point terms)
merged_df["BTC_Huobi_funding_rate"] = merged_df["BTC_Huobi_funding_rate"] / 100
merged_df["BTC_BitMex_funding_rate"] = merged_df["BTC_BitMex_funding_rate"] / 100
merged_df["BTC_OKX_funding_rate"] = merged_df["BTC_OKX_funding_rate"] / 100

In [53]:
# Step 3: Create an interactive plot
fig = go.Figure()

for exchange in exchange_data.keys():
    fig.add_trace(go.Scatter(
        x=merged_df.index,
        y=merged_df[f"{exchange}_funding_rate"],
        mode='lines',
        name=exchange
    ))

fig.update_layout(
    title='BTC Funding Rates Across Exchanges',
    xaxis_title='Date',
    yaxis_title='Funding Rate',
    xaxis=dict(
        tickformat="%b, %Y",  # Display date in Month Day, Year format
        dtick="M1", # Set tick interval to 1 month; for weekly, use "D7"
        tickfont=dict(size=8)
    ),
    yaxis=dict(
        tickformat=".4f"
    ),
)

fig.show()

In [36]:
# Step 4: Save aggregated data into one file
merged_df.to_csv("data/BTC_merged.csv")

### Merge all ETH pairs and graph the funding rates

In [42]:
# Repeat the same process for ETH
exchange_data = {}
for exchange in exchanges:
    coin_exchange = "ETH_" + exchange
    exchange_data[coin_exchange] = read_and_resample_csv("data/" + coin_exchange + ".csv")

In [43]:
# Step 2: Merge data
merged_df_eth = pd.concat(exchange_data, axis=1)
merged_df_eth.columns = [f"{name}_{col}" for name in exchange_data.keys() for col in exchange_data[name].columns]
merged_df_eth.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3264 entries, 2019-11-27 00:00:00 to 2024-05-15 12:00:00
Freq: 12H
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ETH_Binance_funding_rate  3263 non-null   float64
 1   ETH_BitMex_funding_rate   1836 non-null   float64
 2   ETH_Bybit_funding_rate    2605 non-null   float64
 3   ETH_Deribit_funding_rate  2175 non-null   float64
 4   ETH_Huobi_funding_rate    2002 non-null   float64
 5   ETH_OKX_funding_rate      2002 non-null   float64
dtypes: float64(6)
memory usage: 178.5 KB


In [44]:
# Scale back some of the exchanges by 100 (to convert from percentage to point terms)
merged_df_eth["ETH_Huobi_funding_rate"] = merged_df_eth["ETH_Huobi_funding_rate"] / 100
merged_df_eth["ETH_BitMex_funding_rate"] = merged_df_eth["ETH_BitMex_funding_rate"] / 100
merged_df_eth["ETH_OKX_funding_rate"] = merged_df_eth["ETH_OKX_funding_rate"] / 100

In [45]:
# Step 3: Create an interactive plot
fig = go.Figure()

for exchange in exchange_data.keys():
    fig.add_trace(go.Scatter(
        x=merged_df_eth.index,
        y=merged_df_eth[f"{exchange}_funding_rate"],
        mode='lines',
        name=exchange
    ))

fig.update_layout(
    title='ETH Funding Rates Across Exchanges',
    xaxis_title='Date',
    yaxis_title='Funding Rate',
    xaxis=dict(
        tickformat="%b, %Y",  # Display date in Month Day, Year format
        dtick="M1", # Set tick interval to 1 month; for weekly, use "D7"
        tickfont=dict(size=8)
    ),
    yaxis=dict(
        tickformat=".4f"
    ),
)

fig.show()

In [46]:
# Step 4: Save aggregated data into one file
merged_df_eth.to_csv("data/ETH_merged.csv")

### Analyze and compare funding rate trends

In [48]:
merged_df.describe()

,BTC_Binance_funding_rate,BTC_BitMex_funding_rate,BTC_Bybit_funding_rate,BTC_Deribit_funding_rate,BTC_Huobi_funding_rate,BTC_OKX_funding_rate,ETH_Binance_funding_rate,ETH_BitMex_funding_rate,ETH_Bybit_funding_rate,ETH_Deribit_funding_rate,ETH_Huobi_funding_rate,ETH_OKX_funding_rate
count,3419.000000,1836.000000,3024.000000,2322.000000,2002.000000,2002.000000,3263.000000,1836.000000,2605.000000,2175.000000,2002.000000,2002.000000
mean,0.000136,0.008135,0.000155,0.000077,0.008448,0.006308,0.000169,0.006549,0.000180,0.000072,0.007981,0.004349
std,0.000239,0.016644,0.000362,0.000198,0.012145,0.014538,0.000318,0.025786,0.000388,0.000281,0.015648,0.019744
min,-0.001580,-0.291800,-0.002857,-0.001814,-0.058568,-0.096016,-0.003563,-0.600300,-0.003017,-0.003154,-0.150030,-0.276952
25%,0.000043,0.006775,0.000055,-0.000002,0.005201,-0.001620,0.000052,0.006775,0.000057,-0.000010,0.002628,-0.004566
50%,0.000100,0.010000,0.000100,0.000020,0.010000,0.004340,0.000100,0.010000,0.000100,0.000015,0.010000,0.003674
75%,0.000100,0.010000,0.000100,0.000116,0.010000,0.011215,0.000100,0.010000,0.000100,0.000103,0.010000,0.012403
max,0.002490,0.098900,0.003096,0.001759,0.107779,0.130932,0.003353,0.129700,0.003750,0.002296,0.146693,0.147424


In [54]:
merged_df_eth.describe()

,ETH_Binance_funding_rate,ETH_BitMex_funding_rate,ETH_Bybit_funding_rate,ETH_Deribit_funding_rate,ETH_Huobi_funding_rate,ETH_OKX_funding_rate
count,3263.000000,1836.000000,2605.000000,2175.000000,2002.000000,2002.000000
mean,0.000169,0.000065,0.000180,0.000072,0.000080,0.000043
std,0.000318,0.000258,0.000388,0.000281,0.000156,0.000197
min,-0.003563,-0.006003,-0.003017,-0.003154,-0.001500,-0.002770
25%,0.000052,0.000068,0.000057,-0.000010,0.000026,-0.000046
50%,0.000100,0.000100,0.000100,0.000015,0.000100,0.000037
75%,0.000100,0.000100,0.000100,0.000103,0.000100,0.000124
max,0.003353,0.001297,0.003750,0.002296,0.001467,0.001474


In [69]:
# Calculate summary statistics
summary = merged_df.describe().transpose()
summary['exchange'] = summary.index
summary = summary[['exchange', 'mean', 'min', 'max']]

# Annualized funding rates
summary['mean'] = summary['mean'] * 3 * 365

fig = go.Figure()

fig.add_trace(go.Bar(
    x=summary['exchange'],
    y=summary['mean'],
    name="Mean",
    text=summary['mean'] * 100,  # Multiply by 100 to convert to percentage
    texttemplate='%{text:.2f}%',  # Format as percentage with two decimal places
    textposition='inside'  # Position text outside the bars
))
    
fig.update_layout(
    title='Annualized BTC Funding Rates Across Exchanges',
    xaxis_title='Exchange',
    yaxis_title='Annualized Mean Funding Rate',
    barmode='group',  # Display bars side-by-side
    yaxis=dict(
        tickformat=".5f"
    ),
)

fig.show()

In [116]:
# Calculate summary statistics
summary = merged_df_eth.describe().transpose()
summary['exchange'] = summary.index
summary = summary[['exchange', 'mean', 'min', 'max']]

# Annualized funding rates
summary['mean'] = summary['mean'] * 3 * 365

fig = go.Figure()

fig.add_trace(go.Bar(
    x=summary['exchange'],
    y=summary['mean'],
    name="Mean",
    text=summary['mean'] * 100,  # Multiply by 100 to convert to percentage
    texttemplate='%{text:.2f}%',  # Format as percentage with two decimal places
    textposition='inside'  # Position text outside the bars
))

fig.update_layout(
    title='Annualized ETH Funding Rates Across Exchanges',
    xaxis_title='Exchange',
    yaxis_title='Annualized Mean Funding Rate',
    barmode='group',  # Display bars side-by-side
    yaxis=dict(
        tickformat=".2f"
    ),
)

fig.show()

### Number of intervals with negative/positive funding rates

In [76]:
# Count positive and negative intervals
summary_counts = {}

for column in merged_df.columns:
    exchange_name = column
    positive_count = (merged_df[column] >= 0).sum()
    negative_count = (merged_df[column] < 0).sum()
    pos_neg_ratio = positive_count / (positive_count + negative_count)
    summary_counts[exchange_name] = {'positive': positive_count, 'negative': negative_count, 'ratio': pos_neg_ratio}

# Convert summary_counts to DataFrame for better visualization
summary_counts_df = pd.DataFrame.from_dict(summary_counts, orient='index')
summary_counts_df.head(20)

,positive,negative,ratio
BTC_Binance_funding_rate,2999,420,0.877157
BTC_BitMex_funding_rate,1569,267,0.854575
BTC_Bybit_funding_rate,2621,403,0.866733
BTC_Deribit_funding_rate,1670,652,0.719208
BTC_Huobi_funding_rate,1709,293,0.853646
BTC_OKX_funding_rate,1381,621,0.689810


In [78]:
# Count positive and negative intervals
summary_counts_eth = {}

for column in merged_df_eth.columns:
    exchange_name = column
    positive_count = (merged_df_eth[column] >= 0).sum()
    negative_count = (merged_df_eth[column] < 0).sum()
    pos_neg_ratio = positive_count / (positive_count + negative_count)
    summary_counts_eth[exchange_name] = {'positive': positive_count, 'negative': negative_count, 'ratio': pos_neg_ratio}

# Convert summary_counts to DataFrame for better visualization
summary_counts_df = pd.DataFrame.from_dict(summary_counts_eth, orient='index')
summary_counts_df.head()

,positive,negative,ratio
ETH_Binance_funding_rate,2901,362,0.889059
ETH_BitMex_funding_rate,1526,310,0.831155
ETH_Bybit_funding_rate,2270,335,0.871401
ETH_Deribit_funding_rate,1458,717,0.670345
ETH_Huobi_funding_rate,1606,396,0.802198


### Longest periods of negative funding rates

In [105]:
# Step 3: Analyze negative funding rate periods
negative_periods_summary = {}

for column in merged_df.columns:
    exchange_name = column
    negative_periods = (merged_df[column] < 0).astype(int)
    negative_periods_diff = negative_periods.diff().fillna(0).astype(int)
    # Identifying the start and end of negative periods
    starts = negative_periods_diff[negative_periods_diff == 1].index
    ends = negative_periods_diff[negative_periods_diff == -1].index

    # Adjust if the series ends with a negative period
    if len(ends) < len(starts):
        ends = ends.append(pd.Index([negative_periods.index[-1]]))

    # Calculate lengths of negative periods
    periods_lengths = (ends - starts).days * 24
    

    longest_period = periods_lengths.max()
    mean_period = np.mean(periods_lengths.to_numpy())

    negative_periods_summary[exchange_name] = {
        'longest_period_hours': longest_period,
        'mean_period_hours': mean_period,
    }

# Convert to DataFrame for better visualization
negative_periods_summary_df = pd.DataFrame.from_dict(negative_periods_summary, orient='index')
negative_periods_summary_df.head(6)

,longest_period_hours,mean_period_hours
BTC_Binance_funding_rate,192,15.728155
BTC_BitMex_funding_rate,168,16.440945
BTC_Bybit_funding_rate,168,17.806452
BTC_Deribit_funding_rate,552,34.360656
BTC_Huobi_funding_rate,168,10.227273
BTC_OKX_funding_rate,192,11.046575


In [106]:
# Step 3: Analyze negative funding rate periods
negative_periods_summary_eth = {}

for column in merged_df_eth.columns:
    exchange_name = column
    negative_periods = (merged_df_eth[column] < 0).astype(int)
    negative_periods_diff = negative_periods.diff().fillna(0).astype(int)
    # Identifying the start and end of negative periods
    starts = negative_periods_diff[negative_periods_diff == 1].index
    ends = negative_periods_diff[negative_periods_diff == -1].index

    # Adjust if the series ends with a negative period
    if len(ends) < len(starts):
        ends = ends.append(pd.Index([negative_periods.index[-1]]))

    # Calculate lengths of negative periods
    periods_lengths = (ends - starts).days * 24

    longest_period = periods_lengths.max()
    mean_period = np.mean(periods_lengths.to_numpy())

    negative_periods_summary_eth[exchange_name] = {
        'longest_period_hours': longest_period,
        'mean_period_hours': mean_period,
    }

# Convert to DataFrame for better visualization
negative_periods_summary_df_eth = pd.DataFrame.from_dict(negative_periods_summary_eth, orient='index')
negative_periods_summary_df_eth.head(6)

,longest_period_hours,mean_period_hours
ETH_Binance_funding_rate,336,16.883721
ETH_BitMex_funding_rate,120,15.740260
ETH_Bybit_funding_rate,216,18.612245
ETH_Deribit_funding_rate,432,36.184615
ETH_Huobi_funding_rate,336,15.138462
ETH_OKX_funding_rate,576,18.847059


### Check correlations between different exchanges

In [109]:
# Create the correlation matrix
correlation_matrix = merged_df.corr()

# Visualize the correlation matrix as a heatmap
fig = px.imshow(correlation_matrix,
                labels=dict(x="Exchange", y="Exchange", color="Correlation"),
                x=correlation_matrix.columns.str.replace('_funding_rate', ''),
                y=correlation_matrix.index.str.replace('_funding_rate', ''),
                color_continuous_scale='Viridis')

fig.update_layout(title='Correlation Matrix of BTC Funding Rates Across Exchanges',
                  xaxis_title='Exchange',
                  yaxis_title='Exchange')

fig.show()

In [111]:
# Step 3: Create the correlation matrix
correlation_matrix = merged_df_eth.corr()

# Step 4: Visualize the correlation matrix as a heatmap
fig = px.imshow(correlation_matrix,
                labels=dict(x="Exchange", y="Exchange", color="Correlation"),
                x=correlation_matrix.columns.str.replace('_funding_rate', ''),
                y=correlation_matrix.index.str.replace('_funding_rate', ''),
                color_continuous_scale='Viridis')

fig.update_layout(title='Correlation Matrix of ETH Funding Rates Across Exchanges',
                  xaxis_title='Exchange',
                  yaxis_title='Exchange')

fig.show()

### Compare funding rates during bull and bear periods across exchanges

In [120]:
# Step 3: Define bull periods
bull_periods = [
    ("2021-10-01", "2021-12-01"),
    ("2023-11-01", "2024-01-15"),
    ("2024-02-10", "2024-03-12")
]

# Step 4: Calculate means and plot in a bar chart
mean_funding_rates = {}

for start_date, end_date in bull_periods:
    period_df = merged_df.loc[start_date:end_date]
    mean_funding_rates[start_date + " to " + end_date] = period_df.mean() * 3 * 365

mean_funding_rates_df = pd.DataFrame(mean_funding_rates)

fig = go.Figure()

for exchange in mean_funding_rates_df.index:
    fig.add_trace(go.Bar(
        x=mean_funding_rates_df.columns,
        y=mean_funding_rates_df.loc[exchange],
        name=exchange,
        text=mean_funding_rates_df.loc[exchange] * 100,  # Multiply by 100 to convert to percentage
        texttemplate='%{text:.2f}%',  # Format as percentage with two decimal places
        textposition='inside'  # Position text outside the bars
    ))

fig.update_layout(
    title='Annualized Mean BTC Funding Rates During Bull Runs',
    xaxis_title='Bull Period',
    yaxis_title='Mean Funding Rate',
    barmode='group',
    yaxis=dict(
        tickformat=".2f"
    ),
)

fig.show()

In [122]:
# Step 4: Calculate means and plot in a bar chart
mean_funding_rates_eth = {}

for start_date, end_date in bull_periods:
    period_df = merged_df_eth.loc[start_date:end_date]
    mean_funding_rates[start_date + " to " + end_date] = period_df.mean() * 3 * 365

mean_funding_rates_df = pd.DataFrame(mean_funding_rates)

fig = go.Figure()

for exchange in mean_funding_rates_df.index:
    fig.add_trace(go.Bar(
        x=mean_funding_rates_df.columns,
        y=mean_funding_rates_df.loc[exchange],
        name=exchange,
        text=mean_funding_rates_df.loc[exchange] * 100,  # Multiply by 100 to convert to percentage
        texttemplate='%{text:.2f}%',  # Format as percentage with two decimal places
        textposition='inside'  # Position text outside the bars
    ))

fig.update_layout(
    title='Annualized Mean ETH Funding Rates During Bull Runs',
    xaxis_title='Bull Period',
    yaxis_title='Mean Funding Rate',
    barmode='group',
    yaxis=dict(
        tickformat=".2f"
    ),
)

fig.show()

In [130]:
# Step 3: Define bear periods
bear_periods = [
    ("2021-12-15", "2022-12-30"),
    ("2023-08-01", "2023-10-15"),
    ("2024-03-20", "2024-05-05")
]

# Step 4: Calculate means and plot in a bar chart
mean_funding_rates = {}

for start_date, end_date in bear_periods:
    period_df = merged_df.loc[start_date:end_date]
    mean_funding_rates[start_date + " to " + end_date] = period_df.mean() * 3 * 365

mean_funding_rates_df = pd.DataFrame(mean_funding_rates)

fig = go.Figure()

for exchange in mean_funding_rates_df.index:
    fig.add_trace(go.Bar(
        x=mean_funding_rates_df.columns,
        y=mean_funding_rates_df.loc[exchange],
        name=exchange,
        text=mean_funding_rates_df.loc[exchange] * 100,  # Multiply by 100 to convert to percentage
        texttemplate='%{text:.2f}%',  # Format as percentage with two decimal places
        textposition='auto'  # Position text outside the bars
    ))

fig.update_layout(
    title='Annualized Mean BTC Funding Rates During Bear Periods',
    xaxis_title='Bear Period',
    yaxis_title='Mean Funding Rate',
    barmode='group',
    yaxis=dict(
        tickformat=".2f"
    ),
)

fig.show()

In [132]:
# Step 4: Calculate means and plot in a bar chart
mean_funding_rates_eth = {}

for start_date, end_date in bear_periods:
    period_df = merged_df_eth.loc[start_date:end_date]
    mean_funding_rates_eth[start_date + " to " + end_date] = period_df.mean() * 3 * 365

mean_funding_rates_df_eth = pd.DataFrame(mean_funding_rates_eth)

fig = go.Figure()

for exchange in mean_funding_rates_df_eth.index:
    fig.add_trace(go.Bar(
        x=mean_funding_rates_df_eth.columns,
        y=mean_funding_rates_df_eth.loc[exchange],
        name=exchange,
        text=mean_funding_rates_df_eth.loc[exchange] * 100,  # Multiply by 100 to convert to percentage
        texttemplate='%{text:.2f}%',  # Format as percentage with two decimal places
        textposition='auto'  # Position text outside the bars
    ))

fig.update_layout(
    title='Annualized Mean ETH Funding Rates During Bear Periods',
    xaxis_title='Bear Period',
    yaxis_title='Mean Funding Rate',
    barmode='group',
    yaxis=dict(
        tickformat=".2f"
    ),
)

fig.show()